In [1]:
import random

#Pennylane
import pennylane as qml
from pennylane import numpy as np

#PyTorch
import torch
import torchvision
from torchvision import datasets, transforms

In [2]:
dev = qml.device('default.qubit', wires = 6)
n_qubits = 6
n_layer_steps = 5
n_layers_to_add = 2
dim = 2**(n_qubits)

In [3]:
def set_random_gates(n_qubits: int):
    
    gate_set = [qml.RX, qml.RY, qml.RZ]
    chosen_gates = []
    for i in range(n_qubits):
        chosen_gate = random.choice(gate_set)
        chosen_gates.append(chosen_gate)
    return chosen_gates

def norm(complex_vector: np.array) -> float:
    """Returns the norm of a complex vector.
    
    Args:
       complex_vector (np.arrray of shape (dim,)):
           complex vector with an arbitrary dimension.
           
       
    Returns:
        norm (float): norm or magnitude of the complex vector.
            
    """
    
    #define the norm of a complex vector here
    norm = np.sqrt(sum([np.square(np.absolute(complex_vector[i])) for i in range(complex_vector.shape[0])]))
    
    return norm

def random_quantum_state(dim: int, amplitude_type: str = 'complex') -> np.array:
    """Creates a normalized random real or complex vector of a defined
    dimension.
    
    Args:
        dim (int): integer number specifying the dimension
            of the vector that we want to generate.
            
        amplitude_type (str): string indicating if you want to create a
            vector with 'complex' or 'real' number amplitudes.
            
    Returns:
        (np.array): normalized real or complex vector of the given
            dimension.
    """
    #generate an unnormalized complex vector of dimension = dim
    if amplitude_type == 'complex':
        Z = np.random.random(dim) + np.random.random(dim)*1j
    
    #generate an unnormalized real vector of dimension = dim
    elif amplitude_type == 'real':
        Z = np.random.random(dim)
    
    #normalize the complex vector Z
    Z /= norm(Z)
    
    return Z

def density_matrix(target_vector: np.array) -> np.array:
    """Return the density matrix of a pure state.
    
    Arguments:
        target_vector (np.array): Complex vector array.
        
    Returns:
        (np.array): Density matrix of shape (dim, dim),
            where dim is the dimension of the target_vector.
    """
    target_vector_reshape = target_vector.reshape((target_vector.shape[0],1))
    density_matrix = target_vector_reshape @ np.transpose(np.conjugate(target_vector_reshape))
    
    return density_matrix

In [4]:
target_vector = random_quantum_state(dim)
density_matrix = density_matrix(target_vector)

### Templates for Phase I

In [5]:
@qml.template
def apply_layer(gates, weights):
    
    for i in range(n_qubits): 
        gates[i](weights[i], wires = i)
    
    tuples = [(i,i+1) for i in range(n_qubits-1)]

    for tup in tuples:
        qml.CZ(wires=[tup[0], tup[1]])
        
@qml.template #template for non-trainable part of the quantum circuit
def frozen_layers(frozen_layer_gates, frozen_layer_weights): 

    for i in range(len(frozen_layer_gates)):
        apply_layer(frozen_layer_gates[i], frozen_layer_weights[i])

@qml.qnode(dev)
def trainable_circuit(params):
    
    #apply frozen layers
    frozen_layers(layer_gates, layer_weights)
    
    #apply trainable layers
    new_weights = []
    for i in range(n_layers_to_add):
        new_weights.append(params[int(i*n_qubits):int((i+1)*n_qubits)])

    for i in range(n_layers_to_add):
        apply_layer(new_gates[i], new_weights[i])
        
    wirelist = [i for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix, wirelist))

In [6]:
def cost(x):
    
    return (1.0 - trainable_circuit(x))

In [7]:
layer_gates = []
layer_weights = []
n_layers = 2
for i in range(n_layers):
    layer_gates.append(set_random_gates(n_qubits))
    layer_weights.append(np.random.rand(n_qubits)*2*np.pi)
    
new_gates = [set_random_gates(n_qubits) for i in range(n_layers_to_add)]
params = np.random.rand(n_qubits*n_layers_to_add)*2*np.pi

trainable_circuit(params)

0.004151693320899796

In [8]:
layer_gates = []
layer_weights = []
cost_list = []
for step in range(n_layer_steps):
    
    new_gates = [set_random_gates(n_qubits) for i in range(n_layers_to_add)]
    init_params = np.zeros(n_qubits*n_layers_to_add)
    
    opt = qml.GradientDescentOptimizer(stepsize = 0.4)

    opt_steps = 100

    params = init_params
    
    for i in range(opt_steps):

        params = opt.step(cost, params)
    
    
    print(step)
    print(layer_weights)
    print(cost(params))
    cost_list.append(cost(params))
    
    trained_weights = [params[int(i*n_qubits):int((i+1)*n_qubits)] for i in range(n_layers_to_add)]
    
    layer_gates += new_gates
    layer_weights += trained_weights

0
[]
0.9347436721232529
1
[array([ 2.08166817e-17,  2.08166817e-17,  6.77256007e-01, -1.95938131e-02,
        2.84494650e-17,  2.84494650e-17]), array([ 9.55510490e-01,  2.31111593e-33,  1.51334312e-01,  1.44573515e-03,
        6.00726920e-01, -9.71445147e-18])]
0.6769677001837388
2
[array([ 2.08166817e-17,  2.08166817e-17,  6.77256007e-01, -1.95938131e-02,
        2.84494650e-17,  2.84494650e-17]), array([ 9.55510490e-01,  2.31111593e-33,  1.51334312e-01,  1.44573515e-03,
        6.00726920e-01, -9.71445147e-18]), array([-0.00206292, -1.61382242,  0.24949963, -0.08434771,  0.82329436,
        1.51872756]), array([-1.04425733,  0.07071659, -0.38913506, -0.08434771, -0.05089197,
       -0.02357861])]
0.6112466685948622
3
[array([ 2.08166817e-17,  2.08166817e-17,  6.77256007e-01, -1.95938131e-02,
        2.84494650e-17,  2.84494650e-17]), array([ 9.55510490e-01,  2.31111593e-33,  1.51334312e-01,  1.44573515e-03,
        6.00726920e-01, -9.71445147e-18]), array([-0.00206292, -1.61382242, 

In [9]:
layer_weights

[array([ 2.08166817e-17,  2.08166817e-17,  6.77256007e-01, -1.95938131e-02,
         2.84494650e-17,  2.84494650e-17]),
 array([ 9.55510490e-01,  2.31111593e-33,  1.51334312e-01,  1.44573515e-03,
         6.00726920e-01, -9.71445147e-18]),
 array([-0.00206292, -1.61382242,  0.24949963, -0.08434771,  0.82329436,
         1.51872756]),
 array([-1.04425733,  0.07071659, -0.38913506, -0.08434771, -0.05089197,
        -0.02357861]),
 array([ 0.48368434, -0.18598465, -0.78866858, -0.13691897,  0.16321546,
         0.26496746]),
 array([ 0.19830253, -0.04526846, -0.42600979, -0.13691897,  0.06551984,
        -0.02258634]),
 array([-0.05548826,  0.2872001 , -0.47962345, -0.69538605,  0.16560787,
         0.02328975]),
 array([-0.17453665,  0.37532627,  0.21703961,  0.02468493,  0.01037398,
        -0.0643601 ]),
 array([-0.11524228, -0.07583442, -0.19953857,  1.39655321,  0.21680037,
        -0.0667202 ]),
 array([-0.02786299, -0.07583442, -0.19953857, -0.07062471,  0.09061203,
         0.0351

### Phase II

In [10]:
partition_percentage = 0.5
partition_size = int(n_layer_steps*n_layers_to_add*partition_percentage)
n_partition_weights = partition_size*n_qubits
n_sweeps = 2

In [11]:
print(layer_weights[:5])
print(n_partition_weights)
print(partition_size)

[array([ 2.08166817e-17,  2.08166817e-17,  6.77256007e-01, -1.95938131e-02,
        2.84494650e-17,  2.84494650e-17]), array([ 9.55510490e-01,  2.31111593e-33,  1.51334312e-01,  1.44573515e-03,
        6.00726920e-01, -9.71445147e-18]), array([-0.00206292, -1.61382242,  0.24949963, -0.08434771,  0.82329436,
        1.51872756]), array([-1.04425733,  0.07071659, -0.38913506, -0.08434771, -0.05089197,
       -0.02357861]), array([ 0.48368434, -0.18598465, -0.78866858, -0.13691897,  0.16321546,
        0.26496746])]
30
5


In [12]:
#trainable_layers = layer_gates[:partition_size]
#trainable_weights = layer_weights[:partition_size]
#weights_flatten = np.concatenate(trainable_weights).ravel()

#is going to depend on the part of the circuit
@qml.qnode(dev)
def trainable_partition(params):
    
    if partition == 1:
        #apply trainable partition
        weights = []
        for i in range(partition_size):
            weights.append(params[int(i*n_qubits):int((i+1)*n_qubits)])
        
        for i in range(len(layer_gates[:partition_size])):
            apply_layer(layer_gates[:partition_size][i], weights[i])
            
        #apply non-trainable partition
        for i in range(len(layer_gates[partition_size:])):
            apply_layer(layer_gates[partition_size:][i], layer_weights[partition_size:][i])
        
    elif partition == 2:
        #apply non-trainable partition
        for i in range(len(layer_gates[:partition_size])):
            apply_layer(layer_gates[:partition_size][i], layer_weights[:partition_size][i])
    
        #apply trainable partition
        weights = []
        for i in range(partition_size):
            weights.append(params[int(i*n_qubits):int((i+1)*n_qubits)])
        
        for i in range(len(layer_gates[partition_size:])):
            apply_layer(layer_gates[partition_size:][i], weights[i])
            
    wirelist = [i for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix, wirelist))

In [13]:
def cost(x):
    
    return (1.0 - trainable_partition(x))

In [14]:
for sweep in range(n_sweeps):
    
    partition = 1
    trainable_weights = layer_weights[:partition_size]

    init_params = np.concatenate(trainable_weights).ravel()
    
    opt = qml.GradientDescentOptimizer(stepsize = 0.4)

    opt_steps = 100

    params = init_params
    
    for i in range(opt_steps):

        params = opt.step(cost, params)
        
    trained_weights = [params[int(i*n_qubits):int((i+1)*n_qubits)] for i in range(partition_size)]
    
    layer_weights[:partition_size] = trained_weights
    
    print(cost(params))

    partition = 2
    trainable_weights = layer_weights[partition_size:]

    init_params = np.concatenate(trainable_weights).ravel()
    
    opt = qml.GradientDescentOptimizer(stepsize = 0.4)

    opt_steps = 100

    params = init_params
    
    for i in range(opt_steps):

        params = opt.step(cost, params)
        
    trained_weights = [params[int(i*n_qubits):int((i+1)*n_qubits)] for i in range(partition_size)]
    
    layer_weights[partition_size:] = trained_weights
    
    print(cost(params))

0.20263878470089158
0.1862610035506873
0.17612330783315255
0.17220693518854524


### Complete-depth learning

In [15]:
@qml.qnode(dev)
def complete_depth_learning(params):
    
    weights = [params[int(i*n_qubits):int((i+1)*n_qubits)] for i in range(len(layer_gates))]
    
    for i in range(len(layer_gates)):
        apply_layer(layer_gates[i], weights[i])
        
    wirelist = [i for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix, wirelist))

In [16]:
def cost(x):
    
    return (1.0 - complete_depth_learning(x))

In [17]:
init_params = np.zeros(n_qubits*len(layer_gates))
    
opt = qml.GradientDescentOptimizer(stepsize = 0.4)

opt_steps = 200

params = init_params

for i in range(opt_steps):

    params= opt.step(cost, params)
    print(f"\r{cost(params)}", end="")

0.17885304157690285

### Images

In [33]:
import collections
import tensorflow as tf

In [34]:
def reduce_image(x):
    x = tf.reshape(x, [1, 28, 28, 1])
    x = tf.image.resize(x, [4, 4])
    x = tf.reshape(x, [4, 4])
    x = x / 255
    return x.numpy()

def remove_contradicting(xs, ys):
    mapping = collections.defaultdict(set)
    for x, y in zip(xs, ys):
        mapping[str(x)].add(y)

    return zip(*((x, y) for x, y in zip(xs, ys) if len(mapping[str(x)]) == 1))

#Simple implementation of encoding image in a Pennylane's circuit
def convert_to_circuit(image):
    
    values = np.ndarray.flatten(image)
    for i, value in enumerate(values):
        if value > 0.5:
            qml.PauliX(wires = i)

In [35]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_train))

x_train, y_train = zip(*((x, y) for x, y in zip(x_train, y_train) if y in [3, 6]))
x_test, y_test = zip(*((x, y) for x, y in zip(x_test, y_test) if y in [3, 6]))

x_train = [reduce_image(x) for x in x_train]
x_test = [reduce_image(x) for x in x_test]

x_train, y_train = remove_contradicting(x_train, y_train)
x_test, y_test = remove_contradicting(x_test, y_test)

print("Number of filtered training examples:", len(x_train))
print("Number of filtered test examples:", len(x_test))

Number of original training examples: 60000
Number of original test examples: 60000
Number of filtered training examples: 11520
Number of filtered test examples: 1906


In [36]:
x_train[2]

array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.19215687, 0.        ],
       [0.        , 0.        , 0.01568628, 0.        ],
       [0.        , 0.6039216 , 0.19215687, 0.        ]], dtype=float32)